In [1]:
import numpy as np
import gvar as gv
import lsqfit
import sys
import os
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import h5py
import time
from importlib import reload

sys.path.append("../")
from fitter import data_loader as dl
from fitter import bootstrapper as bs
from fitter import fitter as fit
from fitter import special_functions as sf

for j in range(10): # Sometimes this needs to be loaded twice...
    matplotlib.rcParams['figure.figsize'] = [10, 10]

In [3]:
reload(bs)
reload(dl)
reload(fit)
reload(sf)

<module 'fitter.special_functions' from '../fitter/special_functions.py'>

### Title

In [4]:
p_dict = {
    'order' : {
        'fit' : 'nnlo', # 'nlo', 'nnlo', or 'nnnlo'
        'vol' : 0, # max 10
        'include_log' : False, # alphaS
        'include_log2' : True
    }, 
    'fit_type' : 'ma-ratio', #'ma', 'ma-ratio, 'xpt', or 'xpt-ratio'; also 'simultaneous', 'ma-old'
    'F2' : 'FKFK', #'FKFpi', 'FpiFpi', 'FKFK'
    'bias_correct' : True,
    
    'abbrs' : [u'a09m220', u'a09m310', u'a09m350', u'a09m400', u'a12m130',
           u'a12m220',  u'a12m220S', u'a12m220L', u'a12m310', u'a12m350',
           u'a12m400',  u'a15m135XL', u'a15m220', u'a15m310', u'a15m350', 'a15m400'], # u'a15m130'
}

In [19]:
t0 = time.time()

if p_dict['order']['vol'] > 0:
    include_FV = True
else:
    include_FV = False
    
name = p_dict['fit_type'] +'_'+ p_dict['F2']
if include_FV:
    name = name + '_FV'
if p_dict['order']['include_log']:
    name = name + '_alphaS'
if p_dict['order']['include_log2']:
    name = name + '_logSq'

print(name)

# Load data
data_loader = dl.data_loader()
fit_data = data_loader.get_fit_data()


bootstrapper = bs.bootstrapper(fit_data, prior=None, order=p_dict['order'], F2=p_dict['F2'], 
                               fit_type=p_dict['fit_type'], abbrs=p_dict['abbrs'], bias_correct=p_dict['bias_correct'])

new_prior = bootstrapper.create_prior_from_fit()
data_loader.save_prior(new_prior, fit_type=p_dict['fit_type'], F2=p_dict['F2'], 
                       include_FV=include_FV, include_alphaS=p_dict['order']['include_log'], include_logSq=p_dict['order']['include_log2'])

t1 = time.time()


print("\nTotal time (s): ", t1 - t0, "\n")


Total time (s):  0.0003299713134765625 

